# LangChain Expression Language (LCEL)

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_community.llms import Ollama

In [10]:
model = Ollama(model="mistral")
prompt=ChatPromptTemplate.from_template(
    "tell me a fun fact about {topic}"
)
output_parser = StrOutputParser()

In [5]:
chain = prompt | model | output_parser

In [7]:
chain.invoke({"topic": "cats"})

' Sure, I\'d be happy to share a fun fact about cats! 😸 Here it is: Did you know that a group of cats is called a clowder? The term "clowder" comes from the Old English word for "flat piece of land," which is fitting because this is where cats like to gather and hunt. However, it\'s important to note that this term might not be commonly used or recognized by everyone! 😊'

# More complex chain

In [1]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [2]:
vectorstore = DocArrayInMemorySearch.from_texts(
     ["harrison worked at kensho", "bears like to eat honey"],
     embedding = OllamaEmbeddings(model="mxbai-embed-large")
)
retriever = vectorstore.as_retriever()

/Users/hungryfoolish/anaconda3/envs/agent310/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [3]:
retriever.get_relevant_documents("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

this embedding model works much better than mistral for embedding, because mistral is decoder-to-decoder so it's only one directional while mxbai-embed-large is based on BERT

In [4]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [14]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [8]:
from langchain.schema.runnable import RunnableMap

In [15]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [17]:
chain.invoke({"question": "where did harrison work?"})

' Based on the provided context, Harrison worked at Kensho.'

In [18]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [19]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

# Bind

In [20]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [21]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions
model = OllamaFunctions(model="llama3", format="json")

In [22]:
model = model.bind(functions=functions)

In [25]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
runnable = prompt | model

In [26]:
runnable.invoke({"input": "what is the weather in LAX"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'weather_search', 'arguments': '{"airport_code": "LAX"}'}}, id='run-d4001d71-7b26-4e5b-a88c-4be2db41669e-0')

In [27]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [28]:
model = model.bind(functions=functions)

In [29]:
runnable = prompt | model

In [30]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'sports_search', 'arguments': '{"team_name": "Patriots"}'}}, id='run-04a038b7-3007-47ad-a9c1-7376f7b97c87-0')

# Fallbacks

In [31]:
from langchain.llms import Ollama
import json

In [32]:
model = Ollama(model="llama3")
simple_chain = model | json.loads

In [33]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [36]:
model.invoke(challenge)

'Here are three poems in a JSON blob:\n\n```\n[\n  {\n    "title": "Moonlit Whisper",\n    "author": "Luna Nightshade",\n    "firstLine": "As the moon rises high"\n  },\n  {\n    "title": "Whispers in the Wind",\n    "author": "Zephyr Wilder",\n    "firstLine": "The wind carries secrets sweet"\n  },\n  {\n    "title": "Starlight Serenade",\n    "author": "Aurora Starweaver",\n    "firstLine": "In starry skies, I find my voice"\n  }\n]\n```\n\nEach poem is represented as a JSON blob with three properties: `title`, `author`, and `firstLine`.'

In [34]:
simple_chain.invoke(challenge)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [38]:
mistral_model = Ollama(model="mistral", temperature=0)
chain = mistral_model | StrOutputParser() | json.loads
chain.invoke(challenge)

[{'poem': {'title': 'Sonnet of the Seas',
   'author': 'William Shakespeare',
   'firstLine': "Beneath the 'wave's obedience, shines the quiet sea,"}},
 {'poem': {'title': 'The Road Not Taken',
   'author': 'Robert Frost',
   'firstLine': 'Two roads diverged in a yellow wood,'}},
 {'poem': {'title': 'Annabel Lee',
   'author': 'Edgar Allan Poe',
   'firstLine': 'It was many and many a year ago,'}}]

so mistral, even the small version, is better at following format instruction than llama3

In [39]:
final_chain = simple_chain.with_fallbacks([chain])

In [40]:
final_chain.invoke(challenge)

[{'poem': {'title': 'Sonnet of the Seas',
   'author': 'William Shakespeare',
   'firstLine': "Beneath the 'wave's obedience, shines the quiet sea,"}},
 {'poem': {'title': 'The Road Not Taken',
   'author': 'Robert Frost',
   'firstLine': 'Two roads diverged in a yellow wood,'}},
 {'poem': {'title': 'Annabel Lee',
   'author': 'Edgar Allan Poe',
   'firstLine': 'It was many and many a year ago,'}}]